### Tools and Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import random

from collections import defaultdict
import torch
from torch.utils.data import DataLoader, TensorDataset

# Language Models
from transformers import BertTokenizer, BertForMaskedLM, BertModel

# Classification Models
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import re

/Users/owner/miniconda3/envs/ml/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Data

In [3]:
# Read the JSON file of chatracter dialogues
with open("../data/dialogue.json", "r") as file:
    raw_dialogue = json.load(file)

# Create a nested dictionary structure
dialogue = {}

for movie, data in raw_dialogue.items():
    # Initialize movie dictionary if not exists
    if movie not in dialogue:
        dialogue[movie] = {}
        
    # If data is a dictionary (contains character information)
    if isinstance(data, dict):
        for character, lines in data.items():
            if isinstance(lines, list):
                dialogue[movie][character] = lines
    # If data is a list (direct dialogues without character info)
    elif isinstance(data, list):
        dialogue[movie]['unknown'] = data

# Loading moral foundations dictionary
moral_dict = pd.read_csv("../data/mfd_v2.csv")

# Convert DataFrame to dictionary for O(1) lookup
moral_word_dict = dict(zip(moral_dict['word'].str.lower(), moral_dict['category']))

# Group words by category
category_words = defaultdict(list)
for _, row in moral_dict.iterrows():
    category_words[row['category']].append(row['word'])

In [4]:
len(dialogue.keys())

1134

Obtaining speeches with moral foundation word in them

In [5]:
# Optimized storage
moral_dialogue = defaultdict(lambda: defaultdict(list))
moral_dialogue_masked = defaultdict(lambda: defaultdict(list))
ground_truths = defaultdict(lambda: defaultdict(list))

# Precompile regex patterns for all moral words (faster lookup)
moral_word_patterns = {word: re.compile(rf"\b{re.escape(word)}\b") for word in moral_word_dict}

movie_index = 0

# Process dataset efficiently
for movie, characters in dialogue.items():
    movie_index += 1
    for character, lines in characters.items():
        for line in lines:
            lowered_line = line.lower()

            # Tokenize sentence once (to speed up lookup)
            words_in_sentence = set(lowered_line.split())

            # Check if any moral word is in the sentence
            matching_words = words_in_sentence & set(moral_word_dict)

            if matching_words:
                for moral_word in matching_words:
                    pattern = moral_word_patterns[moral_word]
                    
                    # Check if it's a standalone word (more efficient regex search)
                    match = pattern.search(lowered_line)
                    if match:
                        # Store original sentence
                        moral_dialogue[movie][character].append(line)

                        # Mask only the first occurrence of the standalone moral word
                        masked_line = pattern.sub("[MASK]", lowered_line, count=1)
                        moral_dialogue_masked[movie][character].append(masked_line)
                        ground_truths[movie][character].append(moral_word)

                        break  # Stop processing after first match for efficiency

    print("Finished movie ", movie_index)

Finished movie  1
Finished movie  2
Finished movie  3
Finished movie  4
Finished movie  5
Finished movie  6
Finished movie  7
Finished movie  8
Finished movie  9
Finished movie  10
Finished movie  11
Finished movie  12
Finished movie  13
Finished movie  14
Finished movie  15
Finished movie  16
Finished movie  17
Finished movie  18
Finished movie  19
Finished movie  20
Finished movie  21
Finished movie  22
Finished movie  23
Finished movie  24
Finished movie  25
Finished movie  26
Finished movie  27
Finished movie  28
Finished movie  29
Finished movie  30
Finished movie  31
Finished movie  32
Finished movie  33
Finished movie  34
Finished movie  35
Finished movie  36
Finished movie  37
Finished movie  38
Finished movie  39
Finished movie  40
Finished movie  41
Finished movie  42
Finished movie  43
Finished movie  44
Finished movie  45
Finished movie  46
Finished movie  47
Finished movie  48
Finished movie  49
Finished movie  50
Finished movie  51
Finished movie  52
Finished movie  53
Fi

In [6]:
# Convert defaultdict to a normal dictionary for saving
data_to_save = {
    "moral_dialogue": moral_dialogue,
    "moral_dialogue_masked": moral_dialogue_masked,
    "ground_truths": ground_truths
}

# Save as JSON
with open("moral_data.json", "w") as f:
    json.dump(data_to_save, f, indent=4)

How many lines with moral words do we have?

In [7]:
count = 0

for movie in moral_dialogue:
    for character in moral_dialogue[movie]:
        count += len(moral_dialogue[movie][character])

print("Number of lines with moral words: ", count)

Number of lines with moral words:  105046


#### Define function that divides the data

In [8]:
def movie_data_division(split_type="hard", train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    """
    Splits original, masked, and ground truth dialogue data into train, validation, and test sets.

    Args:
        split_type (str): "hard" (by movie), "medium" (by character, randomized), "easy" (by speeches, randomized).
        train_ratio (float): Proportion of data for training.
        val_ratio (float): Proportion of data for validation.
        test_ratio (float): Proportion of data for testing.
        only_moral (bool): If True, only considers dialogues containing moral foundation words.

    Returns:
        tuple: (train_data, train_masked, train_labels), (val_data, val_masked, val_labels), (test_data, test_masked, test_labels)
    """
    assert train_ratio + val_ratio + test_ratio == 1, "Ratios should sum up to 1"

    # Choose the appropriate dataset
    dialogue_data = moral_dialogue
    masked_data = moral_dialogue_masked
    label_data = ground_truths

    train_data, train_masked, train_labels = [], [], []
    val_data, val_masked, val_labels = [], [], []
    test_data, test_masked, test_labels = [], [], []

    if split_type == "hard":
        # Split by movie
        movie_keys = list(dialogue_data.keys())
        random.seed(42)
        random.shuffle(movie_keys)

        total_size = len(movie_keys)
        train_end = int(train_ratio * total_size)
        val_end = train_end + int(val_ratio * total_size)

        train_movies = movie_keys[:train_end]
        val_movies = movie_keys[train_end:val_end]
        test_movies = movie_keys[val_end:]

        for movie in train_movies:
            for character in dialogue_data[movie]:
                train_data.extend(dialogue_data[movie][character])
                train_masked.extend(masked_data[movie][character])
                train_labels.extend(label_data[movie][character])

        for movie in val_movies:
            for character in dialogue_data[movie]:
                val_data.extend(dialogue_data[movie][character])
                val_masked.extend(masked_data[movie][character])
                val_labels.extend(label_data[movie][character])

        for movie in test_movies:
            for character in dialogue_data[movie]:
                test_data.extend(dialogue_data[movie][character])
                test_masked.extend(masked_data[movie][character])
                test_labels.extend(label_data[movie][character])

    elif split_type == "medium":
        # Split by character across all movies
        character_entries = []

        for movie, characters in dialogue_data.items():
            for character, lines in characters.items():
                character_entries.append((movie, character))

        random.seed(42)
        random.shuffle(character_entries)

        total_size = len(character_entries)
        train_end = int(train_ratio * total_size)
        val_end = train_end + int(val_ratio * total_size)

        train_chars = character_entries[:train_end]
        val_chars = character_entries[train_end:val_end]
        test_chars = character_entries[val_end:]

        for movie, character in train_chars:
            train_data.extend(dialogue_data[movie][character])
            train_masked.extend(masked_data[movie][character])
            train_labels.extend(label_data[movie][character])

        for movie, character in val_chars:
            val_data.extend(dialogue_data[movie][character])
            val_masked.extend(masked_data[movie][character])
            val_labels.extend(label_data[movie][character])

        for movie, character in test_chars:
            test_data.extend(dialogue_data[movie][character])
            test_masked.extend(masked_data[movie][character])
            test_labels.extend(label_data[movie][character])

    else:  # "easy" split
        # Split at the individual speech level
        all_speeches = []
        all_masked = []
        all_labels = []

        for movie in dialogue_data:
            for character in dialogue_data[movie]:
                all_speeches.extend(dialogue_data[movie][character])
                all_masked.extend(masked_data[movie][character])
                all_labels.extend(label_data[movie][character])

        combined = list(zip(all_speeches, all_masked, all_labels))
        random.seed(42)
        random.shuffle(combined)

        all_speeches, all_masked, all_labels = zip(*combined)

        total_size = len(all_speeches)
        train_end = int(train_ratio * total_size)
        val_end = train_end + int(val_ratio * total_size)
        test_end = val_end + int(test_ratio * total_size)

        train_data, train_masked, train_labels = all_speeches[:train_end], all_masked[:train_end], all_labels[:train_end]
        val_data, val_masked, val_labels = all_speeches[train_end:val_end], all_masked[train_end:val_end], all_labels[train_end:val_end]
        test_data, test_masked, test_labels = all_speeches[val_end:test_end], all_masked[val_end:test_end], all_labels[val_end:test_end]

    return (train_data, train_masked, train_labels), (val_data, val_masked, val_labels), (test_data, test_masked, test_labels)

In [9]:
back_up_moral_dialogue = moral_dialogue
back_up_moral_dialogue_masked = moral_dialogue_masked
back_up_ground_truths = ground_truths

### Baseline 1 model

We're using the default BERT model (no training)

Hard division

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

# Set model to evaluation mode
model.eval()

# Load validation set from previous data split
(_, _, _), (val_data, val_masked, val_labels), (_, _, _) = movie_data_division(split_type="hard")

def batch_predict_masked_words(masked_sentences, k=5, batch_size=16):
    """Predicts top-k words for multiple [MASK] tokens in batch."""
    tokenized_inputs = tokenizer(masked_sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    dataset = TensorDataset(tokenized_inputs.input_ids, tokenized_inputs.attention_mask)
    dataloader = DataLoader(dataset, batch_size=batch_size)

    predictions = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits  # Shape: [batch_size, seq_length, vocab_size]

            batch_predictions = []
            for i in range(input_ids.shape[0]):  # Iterate over batch items
                mask_indices = torch.where(input_ids[i] == tokenizer.mask_token_id)[0]  # ✅ Find `[MASK]` index per batch item
                
                if len(mask_indices) == 0:  # ✅ If no `[MASK]`, add placeholder prediction
                    batch_predictions.append(["[NO MASK]"] * k)
                    continue
                
                mask_idx = mask_indices[0].item()  # Take first `[MASK]` index
                
                if mask_idx >= logits.shape[1]:  # ✅ Prevent index out of bounds
                    batch_predictions.append(["[NO MASK]"] * k)
                    continue
                
                softmax = torch.nn.functional.softmax(logits[i, mask_idx], dim=0)
                top_k_tokens = torch.topk(softmax, k)
                predicted_words = [tokenizer.decode([token]) for token in top_k_tokens.indices.tolist()]
                batch_predictions.append(predicted_words)

            predictions.extend(batch_predictions)

    return predictions

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls

Plot the accuracy@k for different values of k

In [11]:
# Define range of k values (1 to 10)
top_k_range = list(range(1, 11))
accuracy_scores = []

# Compute Accuracy@k for different values of k
for k in top_k_range:
    top_k_predictions = batch_predict_masked_words(val_masked, k=k, batch_size=16)
    accuracy_at_k = np.mean([1 if truth in pred else 0 for truth, pred in zip(val_labels, top_k_predictions)])
    accuracy_scores.append(accuracy_at_k)

# Plot Accuracy@k
plt.figure(figsize=(8, 5))
plt.plot(top_k_range, accuracy_scores, marker='o', linestyle='-', label='Accuracy@k')

# Labels and title
plt.xlabel("Top-k")
plt.ylabel("Accuracy")
plt.title("Accuracy@k vs. k")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

KeyboardInterrupt: 

In [12]:
def show_utterances(movie = "Beauty and the Beast", character = "BELLE"):
        for i in range(len(moral_dialogue[movie][character])):  # Iterate over sentence indices
            original_sentence = moral_dialogue[movie][character][i]
            masked_sentence = moral_dialogue_masked[movie][character][i]
            ground_truth_word = ground_truths[movie][character][i]

            print(f"🎬 Movie: {movie} | 🎭 Character: {character}")
            print(f"📝 Original: {original_sentence}")
            print(f"🔲 Masked:   {masked_sentence}")
            print(f"✅ Ground Truth: {ground_truth_word}")
            print("-" * 80)  # Separator for readability

In [13]:
show_utterances()

🎬 Movie: Beauty and the Beast | 🎭 Character: BELLE
📝 Original:  To return this book to Pere Robert. It's about two lovers in fair Verona. 
🔲 Masked:    to return this book to pere robert. it's about two lovers in [MASK] verona. 
✅ Ground Truth: fair
--------------------------------------------------------------------------------
🎬 Movie: Beauty and the Beast | 🎭 Character: BELLE
📝 Original:  All I wanted was to teach a child to read. 
🔲 Masked:    all i wanted was to teach a [MASK] to read. 
✅ Ground Truth: child
--------------------------------------------------------------------------------
🎬 Movie: Beauty and the Beast | 🎭 Character: BELLE
📝 Original:  I might be a farm girl, but I'm not simple. I'm sorry, but I will never marry you, Gaston. 
🔲 Masked:    i might be a farm girl, but i'm not simple. i'm sorry, but i will never [MASK] you, gaston. 
✅ Ground Truth: marry
--------------------------------------------------------------------------------
🎬 Movie: Beauty and the Beast | 🎭 C

In [14]:
import random

def show_random_100_utterances():
    # Collect all available sentences with their context (movie, character, index)
    all_sentences = [
        (movie, character, i)
        for movie, characters in moral_dialogue.items()
        for character, sentences in characters.items()
        for i in range(len(sentences))
    ]
    
    # Shuffle and select 100 random samples
    sampled_sentences = random.sample(all_sentences, min(100, len(all_sentences)))
    
    for movie, character, i in sampled_sentences:
        original_sentence = moral_dialogue[movie][character][i]
        masked_sentence = moral_dialogue_masked[movie][character][i]
        ground_truth_word = ground_truths[movie][character][i]
        
        print(f"🎬 Movie: {movie} | 🎭 Character: {character}")
        print(f"📝 Original: {original_sentence}")
        print(f"🔲 Masked:   {masked_sentence}")
        print(f"✅ Ground Truth: {ground_truth_word}")
        print("-" * 80)  # Separator for readability


In [16]:
show_random_100_utterances()

🎬 Movie: All the President's Men | 🎭 Character: WOODWARD
📝 Original:  John Mitchell resigned as head of CREEP to spend more time with his family. That doesn't exactly have the ring of truth. (DEEP THROAT nods) Howard Hunt's been found--there was talk that his lawyer had 25 thousand in cash in a paper bag. 
🔲 Masked:    john mitchell resigned as head of creep to spend more time with his family. that doesn't exactly have the ring of truth. (deep throat nods) howard hunt's been found--there was talk that his [MASK] had 25 thousand in cash in a paper bag. 
✅ Ground Truth: lawyer
--------------------------------------------------------------------------------
🎬 Movie: Bling Ring, The | 🎭 Character: NICKI
📝 Original:  Yeah, his dad has a production company he helps style for. 
🔲 Masked:    yeah, his dad has a production company he [MASK] style for. 
✅ Ground Truth: helps
--------------------------------------------------------------------------------
🎬 Movie: Meet Joe Black | 🎭 Character: JO

In [17]:
def show_random_100_utterances_to_file(output_file="random_utterances.txt"):
    # Collect all available sentences with their context (movie, character, index)
    all_sentences = [
        (movie, character, i)
        for movie, characters in moral_dialogue.items()
        for character, sentences in characters.items()
        for i in range(len(sentences))
    ]
    
    # Shuffle and select 100 random samples
    sampled_sentences = random.sample(all_sentences, min(100, len(all_sentences)))
    
    with open(output_file, "w", encoding="utf-8") as f:
        for movie, character, i in sampled_sentences:
            original_sentence = moral_dialogue[movie][character][i]
            masked_sentence = moral_dialogue_masked[movie][character][i]
            ground_truth_word = ground_truths[movie][character][i]
            
            f.write(f"🎬 Movie: {movie} | 🎭 Character: {character}\n")
            f.write(f"📝 Original: {original_sentence}\n")
            f.write(f"🔲 Masked:   {masked_sentence}\n")
            f.write(f"✅ Ground Truth: {ground_truth_word}\n")
            f.write("-" * 80 + "\n")  # Separator for readability

    print(f"✅ Output written to '{output_file}'")

# Call the function
show_random_100_utterances_to_file()

✅ Output written to 'random_utterances.txt'


In [18]:
def show_random_100_utterances():
    # Collect all available sentences with their context (movie, character, index)
    all_sentences = [
        (movie, character, i)
        for movie, characters in moral_dialogue.items()
        for character, sentences in characters.items()
        for i in range(len(sentences))
    ]
    
    # Shuffle and select 100 random samples
    sampled_sentences = random.sample(all_sentences, min(100, len(all_sentences)))
    
    for idx, (movie, character, i) in enumerate(sampled_sentences, start=1):
        original_sentence = moral_dialogue[movie][character][i]
        print(f"{idx}. ({movie} - {character}): {original_sentence}")

In [19]:
show_random_100_utterances()

1. (Erin Brockovich - ED):  Well, you go threatening to leave, I can't take any chances.  You're the only person around who understands what I'm doing. Things come up, I gotta know I got someone to turn to. 
2. (Nick of Time - HUEY):  Why, looky-here. My customer-quotient just got multiplied by two. What do you think of that?  You'll have to forgive me, sir. Didn't hear your approach. Fact of the matter is, I'm deef as a post. Compliments of the United States Army... (does a snappy salute) ...Artillery Corps. Can I give you a shine, sir? You don't see your face, you don't pay. 
3. (Blade II - REINHARDT):  How many vampires do you think he's killed with this thing? 
4. (Legion - SANDRA):  Somebody help me! Sandra is on the floor with her husband.     Blood pours freely from his neck. Kyle stares at the dying man -- a moment of hesitation -- then he rushes to help. 
5. (His Girl Friday - BURNS):  (to Bruce) You see what I had to put up with? She never trusted me! You argue with her -- ot